# 开了解LCEL
LCEL是LangChain帮助我们快速构建复杂的链式组件的基本语法，并且流处理，并且调用和日志处理都是可以开箱即用的。

## 基本案例开始（prompt+context+output parser）

(动态提示词模板+输出) 这个是最常见也是最基本调的用方式，下面我将用获得城市旅游信息作为案例

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import get_openai_callback

# 提示词模板
prompt = ChatPromptTemplate.from_template("我想去{topic}旅行，我想知道这个地方有什么好玩的")

# gpt模型
model = ChatOpenAI(
    openai_api_key="",
    openai_api_base="",
    temperature=.7
                 )

# 输出模板
print("输出模板:",prompt)

# 输出解析器
output_parser = StrOutputParser()

chain = prompt | model | output_parser

result = chain.invoke({"topic": "云南"})

print("结果:",result)


输出模板: input_variables=['topic'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='我想去{topic}旅行，我想知道这个地方有什么好玩的'))]
结果: 云南是一个充满魅力的地方，有很多值得一游的景点和活动。

1.玉龙雪山：这是云南著名的景点之一，可以乘坐缆车到达山顶，欣赏壮观的雪山风光。

2.丽江古城：古朴的建筑、石板路和美丽的风景让人仿佛穿越回古代。

3.泸沽湖：这是一个美丽的高原湖泊，可以在湖边散步、划船或者骑行。

4.香格里拉：这里有着壮丽的自然风光和独特的藏文化，是探险和徒步旅行的好去处。

5.石林：这是一个被列入世界自然遗产的地方，可以欣赏到奇特的石灰岩地貌景观。

除此之外，云南还有很多名胜古迹、美食和民俗文化等，让人流连忘返。希望你能在云南旅行中度过愉快的时光！


注意下面这个代码，它就是LCEL语法将不同的组件拼凑成一个链

In [2]:
chain = prompt | model | output_parser

**|** 符号类似于Unix管道操作符，它将不同组件链接在一起，将一个组件的输出作为下一个组件的输入。
在这个链条中，用户输入被传递到提示模板，然后提示模板的输出被传递到模型，接着模型的输出被传递到输出解析器。让我们分别看看每个组件，真正理解正在发生的事情。

### 1.Prompt
**prompt**是一个基础prompt模板，它能接受一个变量  **{topic}**，并生成一个最终**prompt_value**。**prompt_value**是一个包装完成的提示的对象，可以传递给LLM（接受字符串作为输入）或ChatModel（接受消息序列作为输入）。它可以与任何一种语言模型类型一起使用，因为它定义了生成**BaseMessages**和生成字符串的逻辑。

In [3]:
# promt模板与参数结合
prompt_value = prompt.invoke({"topic":"云南"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='我想去云南旅行，我想知道这个地方有什么好玩的')])

In [4]:
prompt_value.to_messages()

[HumanMessage(content='我想去云南旅行，我想知道这个地方有什么好玩的')]

In [5]:
prompt_value.to_string()

'Human: 我想去云南旅行，我想知道这个地方有什么好玩的'

### 2.Model
prompt_value 然后传递给模型。在这种情况下，我们的模型是一个 ChatModel，意味着它将输出一个 BaseMessage。

In [6]:
message = model.invoke(prompt_value)
message

AIMessage(content='云南是中国的一个美丽的省份，有着丰富的自然和人文资源。这里有很多好玩的地方，比如：\n\n1. 丽江古城：这是丽江最有名的景点之一，古老的建筑和独特的纳西文化让人流连忘返。\n\n2. 玉龙雪山：这座雪山位于丽江市的北部，是中国最南的雪山之一，雄伟壮观。\n\n3. 三江并流：这里是金沙江、怒江和澜沧江的交汇处，景色壮丽，令人叹为观止。\n\n4. 香格里拉：这是世界上著名的旅游胜地之一，有着壮丽的自然风景和独特的藏传佛教文化。\n\n5. 东巴文化村：这是云南著名的民俗文化村，可以了解到东巴文化的传承和发展。\n\n除此之外，云南还有很多美丽的自然风景和独特的民族文化，例如玉龙雪山、泸沽湖、西双版纳等等，都是值得一游的地方。希望你在云南旅行愉快！')

### 3.output_parser

最后，我们将模型输出传递给output_parser，它是一个BaseOutputParser，意味着它接受字符串或BaseMessage作为输入。StrOutputParser专门将任何输入简单地转换为字符串。

In [7]:
output_parser.invoke(message)

'云南是中国的一个美丽的省份，有着丰富的自然和人文资源。这里有很多好玩的地方，比如：\n\n1. 丽江古城：这是丽江最有名的景点之一，古老的建筑和独特的纳西文化让人流连忘返。\n\n2. 玉龙雪山：这座雪山位于丽江市的北部，是中国最南的雪山之一，雄伟壮观。\n\n3. 三江并流：这里是金沙江、怒江和澜沧江的交汇处，景色壮丽，令人叹为观止。\n\n4. 香格里拉：这是世界上著名的旅游胜地之一，有着壮丽的自然风景和独特的藏传佛教文化。\n\n5. 东巴文化村：这是云南著名的民俗文化村，可以了解到东巴文化的传承和发展。\n\n除此之外，云南还有很多美丽的自然风景和独特的民族文化，例如玉龙雪山、泸沽湖、西双版纳等等，都是值得一游的地方。希望你在云南旅行愉快！'

### 4.完整的Pipeline
以下是完整的步骤：
  1. 我们以所需主题的用户输入作为 {"topic": "云南"} 进行传递。
  2. 提示组件接受用户输入，然后使用该主题构建提示后生成 prompt_value。
  3. 模型组件获取生成的提示，并将其传递给 OpenAI LLM 模型进行回答。模型生成的输出是一个 ChatMessage 对象。
  4. 最后，output_parser 组件接收 ChatMessage，并将其转换成 Python 字符串，在调用方法中返回。

